In [1]:
import os 
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [2]:
from keras.models import *
from keras.layers import Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, core
from keras.layers.merge import concatenate

from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

from keras import backend as keras

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/imp.py", line 342, in load_dynamic
    return _load(s

In [ ]:
# dim of a 3D data: 32x32x28, where 28 is the number of slices
IMG_L = 32
IMG_W = 32
IMG_H = 28

NUM_OF_MODELS = 3
x = Input(shape=(IMG_L, IMG_W, NUM_OF_MODELS*IMG_H))

In [1]:
def cross_model(x):
    x = tf.reshape(x, shape=[IMG_L, IMG_W, NUM_OF_MODELS, IMG_H])
    x = tf.transpose(x, perm=[0, 1, 3, 2])
    x = tf.reshape(x, shape=[IMG_L, IMG_W, NUM_OF_MODELS*IMG_H])
    return x[:, :, :, :10]

In [2]:
y = Lambda(cross_model, output_shape=(IMG_L, IMG_W, NUM_OF_MODELS*IMG_H))(x)
# y = Lambda(cross_model)(x)

NameError: name 'Lambda' is not defined

In [5]:
cross_modelity = Model(x, y)
cross_modelity.summary()
cross_modelity.compile(loss='mse', optimizer='sgd')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 84)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 32, 32, 84)        0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [6]:
img = np.zeros((1, IMG_L, IMG_W, NUM_OF_MODELS*IMG_H), dtype=int)

for i in range(NUM_OF_MODELS):
        img[:, :, :, IMG_H*i:IMG_H*(i+1)] = (i+1)*(np.arange(1,IMG_H+1)+1)
    
img.shape

(1, 32, 32, 84)

Assume we have 3 models, where each model output a 3D image of the size 32x32x28

In [7]:
for i in range(NUM_OF_MODELS):
    print(img[0, 0, 0, IMG_H*i:IMG_H*(i+1)-3])

[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26]
[ 4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48 50 52]
[ 6  9 12 15 18 21 24 27 30 33 36 39 42 45 48 51 54 57 60 63 66 69 72 75 78]


Now, we re-arrange the height of 3 models so that they are vertically flattern

In [8]:
cross_modelity = K.Function(cross_modelity.inputs, cross_modelity.outputs)

In [10]:
print("Input shape", img[0][0].shape)
print(img[0][0])

print("Output shape", cross_modelity([img])[0][0].shape)
print(cross_modelity([img])[0][0])

Input shape (32, 84)
[[ 2  3  4 ..., 81 84 87]
 [ 2  3  4 ..., 81 84 87]
 [ 2  3  4 ..., 81 84 87]
 ..., 
 [ 2  3  4 ..., 81 84 87]
 [ 2  3  4 ..., 81 84 87]
 [ 2  3  4 ..., 81 84 87]]
Output shape (32, 84)
[[  2.   4.   6. ...,  29.  58.  87.]
 [  2.   4.   6. ...,  29.  58.  87.]
 [  2.   4.   6. ...,  29.  58.  87.]
 ..., 
 [  2.   4.   6. ...,  29.  58.  87.]
 [  2.   4.   6. ...,  29.  58.  87.]
 [  2.   4.   6. ...,  29.  58.  87.]]
